# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time
import numpy as np

## 0. Define Region

First, define a region to train a model for.

In [2]:
REGIONS = ["ES","NS","WS","EM","EE","LD","NEE","NWE","SEE","SWE","WAL","WM","YH"]

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [3]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/single_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [4]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, 2:]

In [5]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

In [6]:
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.mean(training_videos[:, :, :, 0])) / (np.std(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.mean(training_videos[:, :, :, 1])) / (np.std(training_videos[:, :, :, 1]))

for n in range(0, len(REGIONS)):
    training_rainfall[:, n] = (training_rainfall[:, n] - np.min(training_rainfall[:, n])) / (np.max(training_rainfall[:, n]) - np.min(training_rainfall[:, n]))

In [7]:
rmses = [
    0.16748,
    0.17988,
    0.17934,
    0.19004,
    0.20176,
    0.17058,
    0.17980,
    0.14958,
    0.18149,
    0.20817,
    0.18044,
    0.20279,
    0.16978
]
rfs = []
for n, r in enumerate(rmses):
    rfs.append(
        (r * (np.max(training_rainfall[:, n]) - np.min(training_rainfall[:, n]))) + np.min(training_rainfall[:, n])
    )

In [8]:
rfs

[0.16748,
 0.17988,
 0.17934,
 0.19004,
 0.20176,
 0.17058,
 0.1798,
 0.14958,
 0.18149,
 0.20817,
 0.18044,
 0.20279,
 0.16978]

In [7]:
print(np.min(training_rainfall, axis=0))
print(np.max(training_rainfall, axis=0))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [8]:
def apply_augmentation(raw_rain, raw_vids):
    vids = np.copy(raw_vids)
    vids = vids + np.random.normal(0, 0.05, vids.shape)
    return np.concatenate((raw_vids, vids), axis=0), np.concatenate((raw_rain, raw_rain), axis=0)

APPLY_AUGMENTATION = True

if APPLY_AUGMENTATION:
    training_videos, training_rainfall = apply_augmentation(training_rainfall, training_videos)

In [9]:
training_videos.shape

(548, 61, 121, 2)

## 2. Model Definition

Next, a CNN model architecture is defined.

In [16]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv2D(32, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third layer
    model.add(Conv2D(8, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [17]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(
        xdata,
        ydata,
        batch_size=8,
        validation_split=0.3,
        callbacks=[tensorboard, es],
        epochs=150
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [18]:
# Flexible parameters
learning_rate = 0.0001

In [19]:
final_errors = {}
# Run model for each region
for ridx, r in enumerate(REGIONS):
    final_errors[r] = {}
    # Run each model multiple times
    for i in range(0, 3):
        run_name = "{}-{}".format(r, int(time.time()))
        tb = TensorBoard(log_dir="D:/PHD_DATA/Video_11-02-2021/SingleFrame/logs/{}".format(run_name))
        model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
        history = train_model(run_name, tb, model, training_videos, training_rainfall[:, ridx],
                              models_folder="D:/PHD_DATA/Video_11-02-2021/SingleFrame/models/")
        final_errors[r][run_name] = history.history["val_loss"][-1]

Epoch 1/150
48/48 [==============================] - 4s 76ms/step - loss: 0.0389 - val_loss: 0.0377
Epoch 2/150
48/48 [==============================] - 2s 41ms/step - loss: 0.0320 - val_loss: 0.0333
Epoch 3/150
48/48 [==============================] - 2s 42ms/step - loss: 0.0282 - val_loss: 0.0334
Epoch 4/150
48/48 [==============================] - 2s 41ms/step - loss: 0.0327 - val_loss: 0.0302
Epoch 5/150
48/48 [==============================] - 2s 41ms/step - loss: 0.0261 - val_loss: 0.0302
Epoch 6/150
48/48 [==============================] - 2s 42ms/step - loss: 0.0270 - val_loss: 0.0292
Epoch 7/150
48/48 [==============================] - 2s 42ms/step - loss: 0.0278 - val_loss: 0.0285
Epoch 8/150
48/48 [==============================] - 2s 42ms/step - loss: 0.0263 - val_loss: 0.0285
Epoch 9/150
48/48 [==============================] - 2s 42ms/step - loss: 0.0229 - val_loss: 0.0282
Epoch 10/150
48/48 [==============================] - 2s 43ms/step - loss: 0.0244 - val_loss: 0.0280

Epoch 31/150
48/48 [==============================] - 2s 43ms/step - loss: 0.0181 - val_loss: 0.0260
Epoch 32/150
48/48 [==============================] - 2s 40ms/step - loss: 0.0232 - val_loss: 0.0258
Epoch 33/150
48/48 [==============================] - 2s 38ms/step - loss: 0.0189 - val_loss: 0.0260
Epoch 34/150
48/48 [==============================] - 2s 40ms/step - loss: 0.0199 - val_loss: 0.0255
Epoch 35/150
48/48 [==============================] - 2s 40ms/step - loss: 0.0204 - val_loss: 0.0256
Epoch 36/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0218 - val_loss: 0.0258
Epoch 37/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0241 - val_loss: 0.0255
Epoch 38/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0237 - val_loss: 0.0254
Epoch 39/150
48/48 [==============================] - 2s 39ms/step - loss: 0.0202 - val_loss: 0.0256
Epoch 40/150
48/48 [==============================] - 2s 40ms/step - loss: 0.0191 - val_los

Epoch 112/150
48/48 [==============================] - 2s 34ms/step - loss: 0.0150 - val_loss: 0.0209
Epoch 113/150
48/48 [==============================] - 2s 36ms/step - loss: 0.0150 - val_loss: 0.0203
Epoch 114/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0144 - val_loss: 0.0201
Epoch 115/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0148 - val_loss: 0.0205
Epoch 116/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0140 - val_loss: 0.0202
Epoch 117/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0168 - val_loss: 0.0202
Epoch 118/150
48/48 [==============================] - 2s 34ms/step - loss: 0.0137 - val_loss: 0.0210
Epoch 119/150
48/48 [==============================] - 2s 38ms/step - loss: 0.0163 - val_loss: 0.0202
Epoch 00119: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_11-02-2021/SingleFrame/models/ES-1613144925.mdl\assets
Epoch 1/150
48/48 [=============================

Epoch 73/150
48/48 [==============================] - 2s 45ms/step - loss: 0.0165 - val_loss: 0.0221
Epoch 74/150
48/48 [==============================] - 2s 47ms/step - loss: 0.0193 - val_loss: 0.0228
Epoch 75/150
48/48 [==============================] - 2s 48ms/step - loss: 0.0183 - val_loss: 0.0223
Epoch 76/150
48/48 [==============================] - 2s 46ms/step - loss: 0.0187 - val_loss: 0.0219
Epoch 77/150
48/48 [==============================] - 2s 44ms/step - loss: 0.0175 - val_loss: 0.0222
Epoch 78/150
48/48 [==============================] - 2s 44ms/step - loss: 0.0162 - val_loss: 0.0223
Epoch 79/150
48/48 [==============================] - 2s 44ms/step - loss: 0.0193 - val_loss: 0.0219
Epoch 80/150
48/48 [==============================] - 2s 48ms/step - loss: 0.0191 - val_loss: 0.0223
Epoch 81/150
48/48 [==============================] - 2s 47ms/step - loss: 0.0158 - val_loss: 0.0225
Epoch 00081: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_11-02-2021

48/48 [==============================] - 3s 52ms/step - loss: 0.0313 - val_loss: 0.0304
Epoch 21/150
48/48 [==============================] - 3s 56ms/step - loss: 0.0272 - val_loss: 0.0299
Epoch 22/150
48/48 [==============================] - 3s 55ms/step - loss: 0.0295 - val_loss: 0.0295
Epoch 23/150
48/48 [==============================] - 3s 56ms/step - loss: 0.0283 - val_loss: 0.0306
Epoch 24/150
48/48 [==============================] - 3s 56ms/step - loss: 0.0313 - val_loss: 0.0293
Epoch 25/150
48/48 [==============================] - 3s 56ms/step - loss: 0.0353 - val_loss: 0.0292
Epoch 26/150
48/48 [==============================] - 3s 57ms/step - loss: 0.0274 - val_loss: 0.0290
Epoch 27/150
48/48 [==============================] - 3s 54ms/step - loss: 0.0299 - val_loss: 0.0297
Epoch 28/150
48/48 [==============================] - 3s 55ms/step - loss: 0.0300 - val_loss: 0.0293
Epoch 29/150
48/48 [==============================] - 3s 56ms/step - loss: 0.0290 - val_loss: 0.0304
Epo

...

48/48 [==============================] - 2s 37ms/step - loss: 0.0364 - val_loss: 0.0376
Epoch 15/150
48/48 [==============================] - 2s 38ms/step - loss: 0.0431 - val_loss: 0.0383
Epoch 16/150
48/48 [==============================] - 2s 38ms/step - loss: 0.0400 - val_loss: 0.0368
Epoch 17/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0422 - val_loss: 0.0385
Epoch 18/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0385 - val_loss: 0.0383
Epoch 19/150
48/48 [==============================] - 2s 38ms/step - loss: 0.0376 - val_loss: 0.0402
Epoch 20/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0400 - val_loss: 0.0371
Epoch 21/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0398 - val_loss: 0.0403
Epoch 00021: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_11-02-2021/SingleFrame/models/SWE-1613147505.mdl\assets
Epoch 1/150
48/48 [==============================] - 4s 69ms/step - 

48/48 [==============================] - 2s 39ms/step - loss: 0.0281 - val_loss: 0.0256
Epoch 57/150
48/48 [==============================] - 2s 39ms/step - loss: 0.0272 - val_loss: 0.0250
Epoch 58/150
48/48 [==============================] - 2s 39ms/step - loss: 0.0248 - val_loss: 0.0247
Epoch 59/150
48/48 [==============================] - 2s 39ms/step - loss: 0.0244 - val_loss: 0.0249
Epoch 60/150
48/48 [==============================] - 2s 40ms/step - loss: 0.0264 - val_loss: 0.0261
Epoch 61/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0248 - val_loss: 0.0248
Epoch 62/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0255 - val_loss: 0.0267
Epoch 63/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0301 - val_loss: 0.0254
Epoch 00063: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_11-02-2021/SingleFrame/models/WAL-1613147584.mdl\assets
Epoch 1/150
48/48 [==============================] - 4s 68ms/step - 

48/48 [==============================] - 2s 35ms/step - loss: 0.0289 - val_loss: 0.0277
Epoch 25/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0325 - val_loss: 0.0273
Epoch 26/150
48/48 [==============================] - 2s 34ms/step - loss: 0.0268 - val_loss: 0.0303
Epoch 27/150
48/48 [==============================] - 2s 37ms/step - loss: 0.0287 - val_loss: 0.0284
Epoch 28/150
48/48 [==============================] - 2s 34ms/step - loss: 0.0299 - val_loss: 0.0275
Epoch 29/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0285 - val_loss: 0.0270
Epoch 30/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0276 - val_loss: 0.0270
Epoch 31/150
48/48 [==============================] - 2s 34ms/step - loss: 0.0310 - val_loss: 0.0280
Epoch 32/150
48/48 [==============================] - 2s 34ms/step - loss: 0.0301 - val_loss: 0.0285
Epoch 33/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0285 - val_loss: 0.0266
Epo

48/48 [==============================] - 2s 35ms/step - loss: 0.0399 - val_loss: 0.0415
Epoch 11/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0365 - val_loss: 0.0411
Epoch 12/150
48/48 [==============================] - 2s 34ms/step - loss: 0.0392 - val_loss: 0.0406
Epoch 13/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0341 - val_loss: 0.0422
Epoch 14/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0377 - val_loss: 0.0388
Epoch 00014: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_11-02-2021/SingleFrame/models/WM-1613147973.mdl\assets
Epoch 1/150
48/48 [==============================] - 4s 68ms/step - loss: 0.0542 - val_loss: 0.0378
Epoch 2/150
48/48 [==============================] - 2s 34ms/step - loss: 0.0340 - val_loss: 0.0347
Epoch 3/150
48/48 [==============================] - 2s 34ms/step - loss: 0.0348 - val_loss: 0.0314
Epoch 4/150
48/48 [==============================] - 2s 35ms/step - loss

48/48 [==============================] - 2s 35ms/step - loss: 0.0249 - val_loss: 0.0279
Epoch 8/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0277 - val_loss: 0.0278
Epoch 9/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0236 - val_loss: 0.0283
Epoch 10/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0258 - val_loss: 0.0279
Epoch 11/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0236 - val_loss: 0.0278
Epoch 12/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0251 - val_loss: 0.0275
Epoch 13/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0247 - val_loss: 0.0279
Epoch 14/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0251 - val_loss: 0.0278
Epoch 15/150
48/48 [==============================] - 2s 36ms/step - loss: 0.0261 - val_loss: 0.0277
Epoch 16/150
48/48 [==============================] - 2s 35ms/step - loss: 0.0220 - val_loss: 0.0277
Epoch

In [20]:
final_errors

{'ES': {'ES-1613144812': 0.02522113360464573,
  'ES-1613144925': 0.02024240233004093,
  'ES-1613145174': 0.02252954989671707},
 'NS': {'NS-1613145369': 0.028879711404442787,
  'NS-1613145444': 0.02817351371049881,
  'NS-1613145487': 0.026597704738378525},
 'WS': {'WS-1613145707': 0.028043610975146294,
  'WS-1613145925': 0.027797851711511612,
  'WS-1613146060': 0.029476307332515717},
 'EM': {'EM-1613146122': 0.03367588669061661,
  'EM-1613146176': 0.03419892117381096,
  'EM-1613146254': 0.03336504474282265},
 'EE': {'EE-1613146320': 0.034235935658216476,
  'EE-1613146396': 0.036271534860134125,
  'EE-1613146475': 0.03416930511593819},
 'LD': {'LD-1613146508': 0.02940172515809536,
  'LD-1613146544': 0.021510083228349686,
  'LD-1613146740': 0.02883271314203739},
 'NEE': {'NEE-1613146786': 0.13307248055934906,
  'NEE-1613146804': 0.028624827042222023,
  'NEE-1613146899': 0.02971583791077137},
 'NWE': {'NWE-1613146957': 0.020050814375281334,
  'NWE-1613147036': 0.019334806129336357,
  'NWE-